# Loading data from Shalek and Satija (2013)

In [ ]:
# Import the flotilla package for biological data analysis
import flotilla

# Import "numerical python" library for number crunching
import numpy as np

# Import "panel data analysis" library for tabular data
import pandas as pd

## Expression data

In [ ]:
%%bash
wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE41nnn/GSE41265/suppl/GSE41265_allGenesTPM.txt.gz

In [ ]:
%%bash
wget http://www.nature.com/nature/journal/v498/n7453/extref/nature12172-s1.zip
unzip nature12172-s1.zip

In [ ]:
expression = pd.read_table("GSE41265_allGenesTPM.txt.gz", compression="gzip", index_col=0)
expression.head()

In [ ]:
expression = expression.T
expression.head()

In [ ]:
singles_ids = expression.index[expression.index.map(lambda x: x.startswith('S'))]
print('number of single cells:', len(singles_ids))
singles = expression.ix[singles_ids]

expression_filtered = expression.ix[:, singles[singles > 1].count() >= 3]
expression_filtered = np.log(expression_filtered + 1)
expression_filtered.shape

## Expression feature data

In [ ]:
expression2 = pd.read_excel('nature12172-s1/Supplementary_Table2.xlsx')

# # This was also in features x samples format, so we need to transpose
# expression2 = expression2.T
# expression2.head()

In [ ]:
set([type(x) for x in expression2.GENE])

In [ ]:
set([x for x in expression2.GENE if isinstance(x, int)])

In [ ]:
import datetime

set([x for x in expression2.GENE if isinstance(x, datetime.datetime)])

In [ ]:
expression2.GENE = expression2.GENE[expression2.GENE.map(lambda x: isinstance(x, unicode))]

In [ ]:
expression2.ix[20:25, :5]

In [ ]:
expression2.shape

In [ ]:
expression2 = expression2.dropna(subset=['GENE'])
expression2.ix[20:25, :5]

In [ ]:
expression2.shape

In [ ]:
expression2.ix[:5, -5:]

In [ ]:
gene_category = expression2[['GENE', 'Gene Category']]
gene_category.head()

In [ ]:
expression_feature_data = gene_category.set_index('GENE')
expression_feature_data.head()

## Splicing Data

In [ ]:
splicing = pd.read_excel('nature12172-s1/Supplementary_Table4.xls', 'splicingTable.txt', index_col=(0,1))
splicing.head()

In [ ]:
splicing = splicing.T
splicing

In [ ]:
splicing.index[splicing.index.map(lambda x: 'P' in x)]

In [ ]:
import re
re.search(r'P\d', '10,000 cell Rep1 (P1)').group()

In [ ]:
def long_pooled_name_to_short(x):
    if 'P' not in x:
        return x
    else:
        return re.search(r'P\d', x).group()


splicing.index.map(long_pooled_name_to_short)

In [ ]:
splicing.index = splicing.index.map(long_pooled_name_to_short)
splicing.head()

In [ ]:
splicing.columns = splicing.columns.droplevel(1)
splicing.head()

### Metadata

In [ ]:
metadata = pd.DataFrame(index=expression_filtered.index)
metadata['phenotype'] = 'BDMC'
metadata['pooled'] = metadata.index.map(lambda x: x.startswith('P'))

metadata

## Mapping stats data

In [ ]:
mapping_stats = pd.read_excel('nature12172-s1/Supplementary_Table1.xls', sheetname='SuppTable1 2.txt')
mapping_stats

## Create a `flotilla` Study!

In [ ]:
study = flotilla.Study(# The metadata describing phenotype and pooled samples
                       metadata, 
                       
                       # A version for this data
                       version='0.1.0', 
                       
                       # Dataframe of the filtered expression data
                       expression_data=expression_filtered,
                       
                       # Dataframe of the feature data of the genes
                       expression_feature_data=expression_feature_data,
                       
                       # Dataframe of the splicing data
                       splicing_data=splicing, 
                       
                       # Dataframe of the mapping stats data
                       mapping_stats_data=mapping_stats, 
                       
                       # Which column in "mapping_stats" has the number of reads
                       mapping_stats_number_mapped_col='PF_READS')

## Save the study so you can load it later

In [ ]:
study.save('shalek2013')

In [ ]:
cat /Users/olga/flotilla_projects/shalek2013/datapackage.json

In [ ]:
ls /Users/olga/flotilla_projects/shalek2013

In [ ]:
study2 = flotilla.embark('shalek2013')